In [1]:
import numpy as np
import pandas as pd
import tweepy
import json
from tweepy import OAuthHandler

In [2]:
consumer_key = "IYJgOYLzBIOtmXbyahCudylfp"
consumer_secret = "QiTgzQLNPzGCyj5VzypNiGD3snMzBRjbQqpJLjOxAhU9WzkkeC"
access_token = "1330559671489290240-YK6Aj5piSqmQ3wi9H4m0MFNTRFeuoJ"
access_token_secret = "3koQzA2MDmhJKCJLAPf1UcmzYteKuUBJCJELKam4ylM3V"

In [3]:
auth = tweepy.OAuthHandler( consumer_key , consumer_secret )
auth.set_access_token( access_token , access_token_secret )
api = tweepy.API(auth)

In [4]:
query = "RCB winning IPL"

In [5]:
Tweets = api.search( query , count=10 , lang='en' , exclude='retweets' , tweet_mode='extended' )
# tweet_mode='extended'
# tweet_mode='compat'

for tweet in Tweets:
    print(tweet,"\n")
    print(".....................................\n")

Status(_api=<tweepy.api.API object at 0x7f227af52898>, _json={'created_at': 'Sun Nov 22 15:10:51 +0000 2020', 'id': 1330529180329549824, 'id_str': '1330529180329549824', 'full_text': "@dhruv_rathee @ArunKum05657718 Like Virat Kohli using firecrackers on cricket matches. Well RCB never used firecrackers on winning IPL 😂.\n\nOr Priyanka Chopra bursting firecrackers on her 'Christian' wedding.", 'truncated': False, 'display_text_range': [31, 207], 'entities': {'hashtags': [], 'symbols': [], 'user_mentions': [{'screen_name': 'dhruv_rathee', 'name': 'Dhruv Rathee 🇮🇳', 'id': 730520573701443584, 'id_str': '730520573701443584', 'indices': [0, 13]}, {'screen_name': 'ArunKum05657718', 'name': 'Ashutosh Kumar', 'id': 1206462904465149952, 'id_str': '1206462904465149952', 'indices': [14, 30]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>', 'in_reply_to_status_id': 132865281355193139

In [6]:
df = pd.DataFrame(columns = ['Tweets' , 'User' , 'User_statuses_count' , 
                            'user_followers' , 'User_location' , 'User_verified' ,
                            'fav_count' , 'rt_count' , 'tweet_date'] )
print(df)
# print(df.shape)

Empty DataFrame
Columns: [Tweets, User, User_statuses_count, user_followers, User_location, User_verified, fav_count, rt_count, tweet_date]
Index: []


In [7]:
def stream(data, file_name):
    i = 0
    for tweet in tweepy.Cursor(api.search, q=data, count=100, lang='en').items():
        print(i, end='\r')
        df.loc[i, 'Tweets'] = tweet.text
        df.loc[i, 'User'] = tweet.user.name
        df.loc[i, 'User_statuses_count'] = tweet.user.statuses_count
        df.loc[i, 'user_followers'] = tweet.user.followers_count
        df.loc[i, 'User_location'] = tweet.user.location
        df.loc[i, 'User_verified'] = tweet.user.verified
        df.loc[i, 'fav_count'] = tweet.favorite_count
        df.loc[i, 'rt_count'] = tweet.retweet_count
        df.loc[i, 'tweet_date'] = tweet.created_at
        df.to_excel('{}.xlsx'.format(file_name))
        i = i+1
        if i == 1000:
            break
        else:
            pass

In [8]:
stream(data=["RCB winning IPL"] , file_name='my_tweets')

In [9]:
df.head()

Tweets  ...           tweet_date
0  @dhruv_rathee @ArunKum05657718 Like Virat Kohl...  ...  2020-11-22 15:10:51
1  RT @Cricbites: This is the winning moment. SRH...  ...  2020-11-22 09:42:28
2  @Its_SakshamR @weirdly_myselff @mufaddal_vohra...  ...  2020-11-21 15:48:04
3  Yaa.. 2021 will be the healing year!!! \nRCB w...  ...  2020-11-20 16:32:40
4  RT @vlp1994: Many years ago, Virat played a cr...  ...  2020-11-20 14:43:38

[5 rows x 9 columns]

In [10]:
from textblob import TextBlob

In [11]:
import re
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [12]:
def analyze_sentiment(tweet):
    analysis = TextBlob(tweet)
    if analysis.sentiment.polarity > 0:
        return 'Positive'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negative'

In [13]:
df['clean_tweet'] = df['Tweets'].apply(lambda x : clean_tweet(x))

In [14]:
df.head()

Tweets  ...                                        clean_tweet
0  @dhruv_rathee @ArunKum05657718 Like Virat Kohl...  ...  rathee Like Virat Kohli using firecrackers on ...
1  RT @Cricbites: This is the winning moment. SRH...  ...  RT This is the winning moment SRH have beaten ...
2  @Its_SakshamR @weirdly_myselff @mufaddal_vohra...  ...  SakshamR myselff vohra As Rohit say mi team no...
3  Yaa.. 2021 will be the healing year!!! \nRCB w...  ...  Yaa 2021 will be the healing year RCB winning ...
4  RT @vlp1994: Many years ago, Virat played a cr...  ...  RT Many years ago Virat played a crucial knock...

[5 rows x 10 columns]

In [15]:
df['Sentiment'] = df['clean_tweet'].apply(lambda x : analyze_sentiment(x) )
df.head()

Tweets  ... Sentiment
0  @dhruv_rathee @ArunKum05657718 Like Virat Kohl...  ...   Neutral
1  RT @Cricbites: This is the winning moment. SRH...  ...  Positive
2  @Its_SakshamR @weirdly_myselff @mufaddal_vohra...  ...  Positive
3  Yaa.. 2021 will be the healing year!!! \nRCB w...  ...  Positive
4  RT @vlp1994: Many years ago, Virat played a cr...  ...  Positive

[5 rows x 11 columns]

In [16]:
n = 1
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 RT @Cricbites: This is the winning moment. SRH have beaten RCB by 6 wickets to qualify for Qualifier 2 against DC. Kane showed his class &amp;…

Clean tweet:
 RT This is the winning moment SRH have beaten RCB by 6 wickets to qualify for Qualifier 2 against DC Kane showed his class amp

Sentiment of the tweet:
 Positive


In [28]:
n = 20
print("Original tweet:\n",df['Tweets'][n])
print()
print("Clean tweet:\n",df['clean_tweet'][n])
print()
print("Sentiment of the tweet:\n",df['Sentiment'][n])

Original tweet:
 Thak gaya hu Vro,,
RCB ko ipl na jeet ta hua dekh dekh ke,,

PS:- its that time of the day when I binge watch RCB i… https://t.co/cwp1KJbBq4

Clean tweet:
 Thak gaya hu Vro RCB ko ipl na jeet ta hua dekh dekh ke PS its that time of the day when I binge watch RCB i

Sentiment of the tweet:
 Neutral


In [24]:
df[df.Sentiment == 'Positive'].shape[0]

34

In [25]:
df[df.Sentiment == 'Neutral'].shape[0]

9

In [26]:
df[df.Sentiment == 'Negative'].shape[0]

10